In [0]:
%restart_python

In [0]:
dbutils.secrets.list('faical-scope')


In [0]:
import os 
from pyspark.dbutils import DBUtils 
d = DBUtils(spark)  
os.environ["API_KEY"] = d.secrets.get(scope="faical-scope", key="API_KEY")


In [0]:
import requests

url = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-tr/records"

all_rows = []
offset = 0

while True:
    # hard stop because Opendatasoft refuses offset >= 10000
    if offset >= 10000:
        print("Reached API offset limit (10000). Cannot fetch more rows with this query.")
        break

    params = {
        # no "limit" here on purpose, we let the API default apply (20)
        "offset": offset,
        "timezone": "Europe/Berlin",
        "refine": [
            'date_heure:"2025"',
            'libelle_region:"Île-de-France"',
        ],
    }

    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    batch = data.get("results", [])

    print(f"offset={offset}, batch_size={len(batch)}")

    if not batch:
        break

    all_rows.extend(batch)
    # advance by the actual batch size (default is 20)
    offset += len(batch)

print("Total fetched:", len(all_rows))


In [0]:
len(all_rows)

In [0]:
from pyspark.sql.types import (
    StructType, StructField,
    StringType, LongType, DoubleType
)

schema = StructType([
    StructField("code_insee_region",   StringType(), True),
    StructField("libelle_region",      StringType(), True),
    StructField("nature",              StringType(), True),
    StructField("date",                StringType(), True),  # YYYY-MM-DD
    StructField("heure",               StringType(), True),  # HH:mm
    StructField("date_heure",          StringType(), True),  # cast to timestamp later

    StructField("consommation",        LongType(),   True),
    StructField("thermique",           LongType(),   True),
    StructField("nucleaire",           LongType(),   True),
    StructField("eolien",              LongType(),   True),
    StructField("solaire",             LongType(),   True),
    StructField("hydraulique",         LongType(),   True),
    StructField("pompage",             StringType(), True),  # values like "0", "-65"
    StructField("bioenergies",         LongType(),   True),
    StructField("ech_physiques",       LongType(),   True),
    StructField("stockage_batterie",   StringType(), True),
    StructField("destockage_batterie", StringType(), True),

    StructField("tco_thermique",       DoubleType(), True),
    StructField("tch_thermique",       DoubleType(), True),
    StructField("tco_nucleaire",       DoubleType(), True),
    StructField("tch_nucleaire",       DoubleType(), True),
    StructField("tco_eolien",          DoubleType(), True),
    StructField("tch_eolien",          DoubleType(), True),
    StructField("tco_solaire",         DoubleType(), True),
    StructField("tch_solaire",         DoubleType(), True),
    StructField("tco_hydraulique",     DoubleType(), True),
    StructField("tch_hydraulique",     DoubleType(), True),
    StructField("tco_bioenergies",     DoubleType(), True),
    StructField("tch_bioenergies",     DoubleType(), True),

    StructField("column_68",           StringType(), True),
])

df = spark.createDataFrame(all_rows, schema=schema)
display(df)


Cast date & datetime 

In [0]:
df.write.format("delta").mode("append").saveAsTable("eco2mix_regional_landing")

In [0]:
df_sql = spark.table("workspace.default.eco2mix_regional_landing")
df_sql.columns